<a href="https://colab.research.google.com/github/alanvgreen/nmigen/blob/pll/experiments/PLLSolver_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copyright 2020 Google LLC.

SPDX-License-Identifier: Apache-2.0

# A Parameter Solver for PLLs

Modern FPGA PLLs can be very sophisticated. However, the core functionality of FPGAs is quite consistent from vendor to vendor and family to family:

- a single input clock
- optional divider on input clock
- a phase detector that drives a VCO to produce the VCO clock
- a VCO clock divider that provides the second input to the phase detector
- multiple output clocks, each with their own divider

This notebook provides a method for calculating the divider parameters for a given FPGA PLL, input clock frequency and output clock frequency.

The ideas here were abstracted from https://github.com/enjoy-digital/litex/blob/master/litex/soc/cores/clock.py, where a less general from of this calculation has been cut-and-paste-and-tweaked into a few diffierent place.

# Clock Frequency.

PLLs (Phase Locked Loops) take a clock of one frequency and produce one or more clocks of other frequencies.

In [ ]:
class FrequencyRange:
  """A range of frequencies. 
  
  May represent a requested clock frequency, an input clock frequency or the
  allowed range of a clock.
  """
  @staticmethod
  def nominally(value, accuracy=0.01):
    return FrequencyRange(value * (1-accuracy), value * (1 + accuracy))

  def __init__(self, min_freq, max_freq):
    self.min_freq = min_freq
    self.max_freq = max_freq

  def  __contains__(self, other):
    """True if the whole range of other is contained in self"""
    return self.min_freq <= other.min_freq and self.max_freq >= other.max_freq

  def __truediv__(self, div):
    """Returns this frequency divided by integer div"""
    return FrequencyRange(self.min_freq / div, self.max_freq / div)
    
  def __mul__(self, mul):
    """Returns this frequency multiplied by integer mul"""
    return FrequencyRange(self.min_freq * mul, self.max_freq * mul)

  def __repr__(self):
    return f"FrequencyRange({self.min_freq}, {self.max_freq})"
    
FrequencyRange.nominally(10) in FrequencyRange(8, 400)

True

In [ ]:
class Problem:
  def __init__(self, input_freq, output_freqs):
    self.input_freq = input_freq
    self.output_freqs = output_freqs
  def __repr__(self):
    return f"Problem({self.input_freq}, {self.output_freqs})"

class Solution:
  def __init__(self, input_div, vco_div, output_divs):
    self.input_div = input_div
    self.vco_div = vco_div
    self.output_divs = output_divs

  def __repr__(self):
    return f"Solution({self.input_div}, {self.vco_div}, {self.output_divs})"


In [ ]:
class Solver:
  def __init__(self, 
               allowed_input_divs, 
               allowed_vco_divs, 
               allowed_output_divs, 
               allowed_input_range,
               allowed_pfd_range, 
               allowed_vco_range, 
               allowed_output_range):
    self.allowed_input_divs = allowed_input_divs
    self.allowed_vco_divs = allowed_vco_divs
    self.allowed_output_divs = allowed_output_divs
    self.allowed_input_range = allowed_input_range
    self.allowed_pfd_range = allowed_pfd_range
    self.allowed_vco_range = allowed_vco_range
    self.allowed_output_range = allowed_output_range

  def solve(self, problem):
    # Check input params
    assert problem.input_freq in self.allowed_input_range
    for o in problem.output_freqs: assert o in self.allowed_output_range

    for input_div in self.allowed_input_divs:
      # Try a PFD frequency
      pfd_freq = problem.input_freq / input_div
      if pfd_freq in self.allowed_pfd_range:
        for vco_div in self.allowed_vco_divs:
          # Try a VCO frequency
          vco_freq = pfd_freq * vco_div
          if vco_freq in self.allowed_vco_range:
            # Check whether every output can be made
            output_divs = []
            vco_freq_is_ok = True
            for output_freq in problem.output_freqs:
              if vco_freq_is_ok:
                for output_div in self.allowed_output_divs:
                  if (vco_freq / output_div) in output_freq:
                    output_divs.append(output_div)
                    break
                else:
                  vco_freq_is_ok = False
            if vco_freq_is_ok:
              yield Solution(input_div, vco_div, output_divs)

  def solve1(self, problem):
    return next(solve(self, problem), None)
            
def closed_range(min, max): return range(min, max+1)

ECP5_SOLVER = Solver(
    closed_range(1, 128),
    closed_range(1, 128),
    closed_range(1, 128),
    FrequencyRange(8.0, 400.0),
    FrequencyRange(10.0, 400.0),
    FrequencyRange(400.0, 800.0),
    FrequencyRange(3.125, 400.0)
)


def print_all_solutions(problem):
  solutions = [s for s in ECP5_SOLVER.solve(problem)]
  print(f"solve({problem})")
  print(f"  There are {len(solutions)} solutions.")
  for solution in solutions:
    print(f"    {solution}")

one_clock = Problem(
   FrequencyRange.nominally(36, 0.01),
   [FrequencyRange.nominally(224, 0.015)])

three_clocks = Problem(
    FrequencyRange.nominally(50, 0.01),
    [FrequencyRange.nominally(100, 0.05), FrequencyRange.nominally(198 ,0.05), FrequencyRange.nominally(70, 0.05)])

print_all_solutions(one_clock)
print_all_solutions(three_clocks)

solve(Problem(FrequencyRange(35.64, 36.36), [FrequencyRange(220.64, 227.35999999999999)]))
  There are 2 solutions.
    Solution(2, 25, [2])
    Solution(3, 56, [3])
solve(Problem(FrequencyRange(49.5, 50.5), [FrequencyRange(95.0, 105.0), FrequencyRange(188.1, 207.9), FrequencyRange(66.5, 73.5)]))
  There are 6 solutions.
    Solution(2, 31, [8, 4, 11])
    Solution(3, 37, [6, 3, 9])
    Solution(3, 47, [8, 4, 11])
    Solution(4, 49, [6, 3, 9])
    Solution(4, 62, [8, 4, 11])
    Solution(4, 63, [8, 4, 11])
